In [1]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/날씨 콘테스트/EDA_data.csv')


In [2]:
df.isnull().sum()

Unnamed: 0           0
Unnamed: 0.1         0
sex                  0
frequency        12444
YMD                  0
area                 0
3시간기온                0
6시간강수량           10000
6시간신적설           10000
강수형태                 0
강수확률                 0
습도                   0
일최고기온                0
일최저기온                0
풍속                   0
풍향                   0
하늘상태              6256
month                0
season               0
day                  0
is_it_weekend        0
dtype: int64

In [3]:
df.drop(columns=['Unnamed: 0','Unnamed: 0.1'] , inplace=True) 


In [4]:
# 2015, 2016 분리
index_2015 = df[df['YMD'] == '2015-01-01'].index[0] 
index_2016 = df[df['YMD'] == '2016-01-01'].index[0]

df_train = df.iloc[:index_2015, : ] # 2012-2014
df_val = df.iloc[index_2015:index_2016, : ] # 2015
df_test = df.iloc[index_2016:, : ] # 2016

In [5]:
print(df_train.shape, df_val.shape, df_test.shape) # 예보 데이터


(35190, 19) (12070, 19) (12444, 19)


In [6]:
# 결측치 비율 확인
def null_per(df):
  
  return df.isnull().sum() / len(df) * 100 

print(null_per(df_train), '\n') # 6시간 강수량, 적설량, 풍속 결측치 많은데 imputer 쓸 예정
print(null_per(df_val), '\n')
print(null_per(df_test))

sex               0.000000
frequency         0.000000
YMD               0.000000
area              0.000000
3시간기온             0.000000
6시간강수량           28.417164
6시간신적설           28.417164
강수형태              0.000000
강수확률              0.000000
습도                0.000000
일최고기온             0.000000
일최저기온             0.000000
풍속                0.000000
풍향                0.000000
하늘상태             14.782609
month             0.000000
season            0.000000
day               0.000000
is_it_weekend     0.000000
dtype: float64 

sex              0.0
frequency        0.0
YMD              0.0
area             0.0
3시간기온            0.0
6시간강수량           0.0
6시간신적설           0.0
강수형태             0.0
강수확률             0.0
습도               0.0
일최고기온            0.0
일최저기온            0.0
풍속               0.0
풍향               0.0
하늘상태             0.0
month            0.0
season           0.0
day              0.0
is_it_weekend    0.0
dtype: float64 

sex                0.000000
frequency        100.00000

In [7]:
from datetime import datetime, date, time
## 일단 YMD로 feature 진행

df_train['YMD'] = pd.to_datetime(df_train['YMD'], format='%Y-%m-%d')
df_val['YMD'] = pd.to_datetime(df_val['YMD'], format='%Y-%m-%d')
df_test['YMD'] = pd.to_datetime(df_test['YMD'], format='%Y-%m-%d')

# df_train['year'] = df_train['YMD'].dt.year # year은 중요한 피쳐가 아닌듯..


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [8]:
def is_it_day1(day):
  if day == 1:
    return 'Yes'
  else : 
    return 'No'

df_train['is_it_day1'] = df_train['YMD'].dt.day.apply(is_it_day1) 
df_val['is_it_day1'] = df_val['YMD'].dt.day.apply(is_it_day1) 
df_test['is_it_day1'] = df_test['YMD'].dt.day.apply(is_it_day1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [9]:
df_list = [df_train, df_val, df_test]


In [10]:
# 지역별 1인당 개인소득
income_dic = {'강원':20210, '경기':21145, '경남':19556, '경북':19620, '광주':21286, '대구':20087, '대전':21351, 
              '부산':20387, '서울':24061, '세종':21191, '울산':23559,'인천':20132, 
              '전남':20103, '전북':20023, '제주':19921, '충남':20011	, '충북':19821}

def income(col):
  return income_dic[col]

In [11]:
for i in df_list:
  i['income'] = i['area'].apply(income)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
# 지역별 인구수 2014 기준
population_dic = {'강원':1544442	, '경기':12357830, '경남':3350257	, '경북':2700794, '광주':1475884, '대구':2493264	,
                  '대전':1531809	, '부산':3519401	, '서울':10103233	, '세종':156125, '울산':1166377,'인천':2902608, 
              '전남':1905780, '전북':1871560	, '제주':607346, '충남':2062273	, '충북':1578933}

def popul(col):
  return population_dic[col]


In [13]:
for i in df_list:
  i['population'] = i['area'].apply(popul)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
for i in df_list:
  i['sex'] = i['sex'].astype(str)
  i['month'] = i['month'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
for i in df_list:
  i.drop(columns=['YMD', '6시간강수량', '6시간신적설', '하늘상태'], inplace=True)

  # df.drop(columns=['6시간강수량', '6시간신적설', '풍속', '하늘상태'])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [16]:
# 타겟 이상치(outlier)를 제거
# df_train = df_train[df_train['frequency'] < 7] # val은 테스트 해야하니 놉! 


In [17]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [18]:
# val set split
from sklearn.model_selection import train_test_split

target = 'frequency'
features = df_train.columns.drop(target) # 이미 array type

X_train, y_train = df_train.loc[:, features], df_train.loc[:, target]
X_val, y_val = df_val.loc[:, features], df_val.loc[:, target]
X_test, y_test = df_test.loc[:, features], df_test.loc[:, target]

In [19]:
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(35190, 17) (12070, 17) (12444, 17) (35190,) (12070,) (12444,)


# model


In [20]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 6.4 MB/s 


In [21]:
from numpy import NaN
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor 
from sklearn.preprocessing import StandardScaler, Normalizer, RobustScaler, MinMaxScaler

from sklearn.compose import TransformedTargetRegressor

from sklearn.metrics import mean_squared_error 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [22]:
pipe = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    SimpleImputer(strategy='mean'), 
    StandardScaler(),
    # RobustScaler(with_centering=True, with_scaling=True, quantile_range=(0.0, 75.0), copy=True),
    # MinMaxScaler(),
   XGBRegressor(alpha=10, booster='gblinear',
                              colsample_bytree=0.3, learning_rate=0.2,
                              max_depth=9, n_estimators=800, n_jobs=-1,
                              objective='reg:squarederror', reg_lambda=0,
                              validate_parameters=1)
)

In [23]:
# 모델 평가

pipe.fit(X_train, y_train)
y_pred_train = pipe.predict(X_train) # predict() - Transform the data, and apply predict with the final estimator.
y_pred_val = pipe.predict(X_val) # transform - 학습하고, 최종 모델로 데이터 변환

RMSE_train = mean_squared_error(y_train, y_pred_train, squared=False) # squared=False - False 면 RRMSE 
RMSE_val = mean_squared_error(y_val, y_pred_val, squared=False) # squared=False - False 면 RRMSE 

print('기본 모델 train RMSE: ', RMSE_train)
print('기본 모델 val RMSE: ', RMSE_val)

기본 모델 train RMSE:  1.2202734924402305
기본 모델 val RMSE:  1.2475784435661361


In [22]:
fi

In [30]:
# 최종 모델로 결과 예측 후 
y_pred_final = pipe.predict(X_test)


In [31]:
### 결과값이 float인데 정수로 예측해야 하나?? -> 이거 드디어 한다
# 결과값 만져주기

df_pred = pd.Series(y_pred_final)
df_pred.describe()

count    12444.000000
mean         1.396579
std          1.131090
min         -0.346455
25%          0.698579
50%          1.106759
75%          1.763046
max          5.783256
dtype: float64

Fitting 3 folds for each of 50 candidates, totalling 150 fits


In [32]:
# numpy.ndarray -> 시리즈로 변환 후 내보내기
pd.Series(y_pred_final).to_csv('/content/drive/MyDrive/Colab Notebooks/날씨 콘테스트/final_pred.csv', index=False )


# extra

In [ ]:
XGBRegressor(base_score=0.5, objective='reg:squarederror', booster='gblinear',
              colsample_bytree=0.3, 
              importance_type='gain', 
             learning_rate=0.15,  max_depth=9,
             min_child_weight=10, 
             n_estimators=100, n_jobs=16, 
             random_state=0, alpha = 10, reg_lambda=0, scale_pos_weight=1,
              validate_parameters=1,
             )

In [ ]:
# 이상치 계산
def cal_outlier 
q3 = df.quantile(0.75) # df['국어'].quantile(0.75) 처럼 특정 열만 적용 가능
q1 = df.quantile(0.25)

iqr = q3 - q1

In [ ]:
for i in df_train.columns:
  if df_train[i].dtype == float:
    print(df_train[i].describe())

>  rmse
- 그냥, 1.2720579911628662